&nbsp;
&nbsp;
![](../_resources/images/e2e-5.png)


# Generative AI with Databricks

## From Predictive to Prescriptive Maintenance
Manufacturers face labor shortages, supply chain disruptions, and rising costs, making efficient maintenance essential. Despite investments in maintenance programs, many struggle to boost asset productivity due to technician shortages and poor knowledge-sharing systems. This leads to knowledge loss and operational inefficiencies.

<div style="font-family: 'DM Sans';">
  <div style="width: 400px; color: #1b3139; margin-left: 50px; margin-right: 50px; float: left;">
    <div style="color: #ff5f46; font-size:50px;">73%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers struggle to recruit maintenance technicians — McKinsey (2023)
    </div>
    <div style="color: #ff5f46; font-size:50px;">55%</div>
    <div style="font-size:25px; margin-top: -20px; line-height: 30px;">
      of manufacturers lack formal knowledge-sharing systems — McKinsey (2023)
    </div>
  </div>
</div>

Generative AI can transform maintenance by reducing downtime and improving productivity. While predictive maintenance anticipates failures, Generative AI enables prescriptive maintenance. Using historical data, AI systems can identify issues, generate solutions, and assist technicians, allowing junior staff to perform effectively and freeing experts for complex tasks.
<br><br>

### From Models to Agent Systems
Generative AI is moving from standalone models to modular agent systems ([Zaharia et al., 2024](https://bair.berkeley.edu/blog/2024/02/18/compound-ai-systems/)). These systems integrate retrievers, models, prompts, and tools to handle complex tasks. Their modular design allows seamless upgrades (e.g., integrating a new LLM) and adaptation to changing needs.

<br><br>
<img style="float: right; margin-top: 10px;" width="700px" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/team_flow_liza.png" />


<br>
<div style="font-size: 19px; margin-left: 0px; clear: left; padding-top: 10px; ">
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="float: left;" width="80px"> 
<h3 style="padding: 10px 0px 0px 5px;">Liza, a Generative AI engineer, uses the Databricks Intelligence Platform to:</h3>
<ul style="list-style: none; padding: 0; margin-left: 05%;">
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">1</div>
    Build real-time data pipelines
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">2</div>
    Retrieve vectors & features
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">3</div>
    Create AI agent tools
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">4</div>
    Build & deploy agents
  </li>
  <li style="margin-bottom: 10px; display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">5</div>
    Operate in batch or real-time
  </li>
  <li style="display: flex; align-items: center;">
    <div class="badge" style="height: 30px; width: 30px; border-radius: 50%; background: #fcba33; color: white; text-align: center; line-height: 30px; font-weight: bold; margin-right: 10px;">6</div>
    Evaluate agent performance
  </li>
</ul>
</div>

**Databricks empowers Liza with a Data + AI platform for Prescriptive Maintenance.** Let’s explore how to deploy this in production.

<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2F05-Generative-AI%2F05.1-ai-tools-iot-turbine-prescriptive-maintenance&version=1">

## Building Agent Systems with Databricks Mosaic AI agent framework

We will build an Agent System designed to generate prescriptive work orders for wind turbine maintenance technicians. This system integrates multiple interacting components to ensure proactive and efficient maintenance, thereby optimizing the overall equipment effectiveness.

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_0.png" style="margin-left: 5px; float: right"  width="1000px;">

Databricks simplifies this by providing a built-in service to:

- Create and store your AI tools leveraging UC functions
- Execute the AI tools in a safe way
- Use agents to reason about the tools you selected and chain them together to properly answer your question. 


This notebook creates the three Mosaic AI tools and associated Mosaic AI endpoints, which will be composed together into a agent in notebook [05.2-agent-creation-guide]($./05.2-agent-creation-guide).
1. **Turbine predictor** which uses a Model Serving endpoint to predict turbines at risk of failure.
2. **Turbine specifications retriever** which retrieve the turbine specifications based on its id.
3. **Turbine maintenance guide**  which uses a Vector Search endpoint to retrieve maintenance guide based on the turbines and issues being adressed.

In [0]:
%pip install databricks-vectorsearch==0.49 databricks-feature-engineering==0.8.0 databricks-sdk==0.40.0 
dbutils.library.restartPython()

  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Not uninstalling protobuf at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8576c285-4c5b-4e45-b41d-ba522ba4a4da
    Can't uninstall 'protobuf'. No files were found to uninstall.
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Not uninstalling click at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8576c285-4c5b-4e45-b41d-ba522ba4a4da
    Can't uninstall 'click'. No files were found to uninstall.
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
    Not uninstalling blinker at /usr/lib/python3/dist-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-8576c285-4c5b-4e45-b41d-ba522ba4a4da
    Can't uninstall 'blinker'. No files were found to uninstall.
ERROR: pip's dependency resolver does not cur

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## Configuration file

Please change your catalog and schema here to run the demo on a different catalog.

 
<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=4003492105941350&notebook=%2Fconfig&demo_name=lakehouse-iot-platform&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-iot-platform%2Fconfig&version=1">


# Technical Setup notebook. Hide this cell results
Initialize dataset to the current user and cleanup data when reset_all_data is set to true

Do not edit

USE CATALOG `main`
using catalog.database `main`.`dbdemos_iot_turbine`


data already existing. Run with reset_all_data=true to force a data cleanup for your local demo.


In [0]:
import pandas as pd 
import numpy as np

## Part 1: Create the Turbine Predictor as a tool to predict turbine failure

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_1.png" style="float: right; width: 600px; margin-left: 10px">

To enable our Agent System to predict turbine failtures based on industrial IoT sensor readings, we will rely on the model we deployed previously in the  [./04.3-running-inference-iot-turbine]($./04.3-running-inference-iot-turbine) notebook. 

**Make sure you run this ML notebook to create the model serving endpoint!**


### Using the Model Serving as tool to predict faulty turbines
Let's define the turbine predictor tool function our LLM agent will be able to execute. 

AI agents use [AI Agent Tools](https://docs.databricks.com/en/generative-ai/create-log-agent.html#create-ai-agent-tools) to perform actions besides language generation, for example to retrieve structured or unstructured data, execute code, or talk to remote services (e.g. send an email or Slack message). 

These functions can contain any logic, from simple SQL to advanced python. Below we wrap the model serving endpoint in a SQL function using '[ai_query function](https://docs.databricks.com/en/sql/language-manual/functions/ai_query.html)'.

In [0]:

API_ROOT = f"https://{dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().value()}/"

API_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset,  api_root=API_ROOT, api_token=API_TOKEN, model_serving_endpoint_name='dbdemos_iot_turbine_prediction_endpoint'):

# Get the API endpoint and token for the current notebook context

    url = f'{api_root}/serving-endpoints/{model_serving_endpoint_name}/invocations'

    headers = {'Authorization': f'Bearer {api_token}', 
               'Content-Type': 'application/json'}


    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)

    data_json = json.dumps(ds_dict, allow_nan=True)

    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()



In [0]:

import json
import requests

columns = ['avg_energy', 'std_sensor_A', 'std_sensor_B', 'std_sensor_C', 'std_sensor_D', 'std_sensor_E', 'std_sensor_F']

samp_ar = np.array([[0.1889792 , 0.9644652 , 2.65583866, 3.4528106 , 2.48515875,
        2.28840325, 4.70213899]])

samp_df = pd.DataFrame(samp_ar, columns=columns)

score_model(samp_df)


{'predictions': [0]}

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, DoubleType


def turbine_maintenance_predictor_py(sensor_values):
    # Simulate the ai_query function
    
    prediction = score_model(sensor_values)
    
    if prediction['predictions'][0] == 0:
        return 'Sensor F fault'
    elif prediction['predictions'][0] == 1:
        return 'ok'
    elif prediction['predictions'][0] == 2:
        return 'Sensor B fault'
    elif prediction['predictions'][0] == 3:
        return 'Sensor D fault'
    else:
        return 'faulty'

# Register the UDF
# spark.udf.register("turbine_maintenance_predictor_py", turbine_maintenance_predictor_py, StringType())


In [0]:
%sql
CREATE OR REPLACE FUNCTION main.dbdemos_iot_turbine.turbine_maintenance_predictor(sensor_values ARRAY<DOUBLE>)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'This tool predicts whether or not a turbine is faulty to facilitate proactive maintenance. It expects an array of 7 double values (energy and sensor readings) as input and returns a string indicating if a particular sensor is predicted to be faulty or if all sensors are ok.'
AS 
$$

import numpy as np
import pandas as pd
import json 
import requests

#API TOKEN AND URL HERE
api_token = '<your-api-token-here>'
api_root = 'https://<your-api-root-here>.cloud.databricks.com'

model_serving_endpoint_name = 'dbdemos_iot_turbine_prediction_endpoint'

columns = ['avg_energy', 'std_sensor_A', 'std_sensor_B', 'std_sensor_C', 'std_sensor_D', 'std_sensor_E', 'std_sensor_F']

samp_ar = np.array([sensor_values])

data = pd.DataFrame(samp_ar, columns=columns)

url = f'{api_root}/serving-endpoints/{model_serving_endpoint_name}/invocations'

headers = {'Authorization': f'Bearer {api_token}', 
            'Content-Type': 'application/json'}


ds_dict = {'dataframe_split': data.to_dict(orient='split')} if isinstance(data, pd.DataFrame) else tf_serving_json

data_json = json.dumps(ds_dict, allow_nan=True)

response = requests.request(method='POST', headers=headers, url=url, data=data_json)

if response.status_code != 200:
    raise Exception(f'Request failed with status {response.status_code}, {response.text}')

if response.json()['predictions'][0] == 0:
    return 'Sensor F fault'
elif response.json()['predictions'][0] == 1:
    return 'ok'
elif response.json()['predictions'][0] == 2:
    return 'Sensor B fault'
elif response.json()['predictions'][0] == 3:
    return 'Sensor D fault'
else:
    return 'faulty'

$$ 


In [0]:
%sql
select
turbine_maintenance_predictor(array(0.1889792, 
0.9644652, 
2.65583866, 
3.4528106, 
2.48515875,
2.28840325, 4.70213899)) as prediction



prediction
Sensor F fault


In [0]:
# %sql
# -- for testing purposes only
# DROP FUNCTION IF EXISTS get_dummy_sensor_value_readings;

# CREATE OR REPLACE FUNCTION get_dummy_sensor_value_readings()
# RETURNS ARRAY<DOUBLE>
# LANGUAGE SQL
# COMMENT 'Returns an array of 7 random double values simulating sensor readings'
# RETURN (
#   array(
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand(), 
#     rand()
#   )
# );

In [0]:
# df = spark.sql("SELECT get_dummy_sensor_value_readings() AS sensor_values")
# display(df)

Agent should call turbine_specifications_retriever() to get sensor readings, then agent needs to format readings as an array and call turbine_maintenance_predictor() to get a prediction

Now we can test out our function below:

## Part 3: Add a tool to access our maintenance guide content and provide support to the operator during maintenance operation

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/manufacturing/lakehouse-iot-turbine/iot_agent_graph_v2_3.png" style="float: right; width: 600px; margin-left: 10px">


We were provided with PDF guide containing all the error code and maintenance steps for the critical components of our wind turbine. The're saved as pdf file in our volume.

Let's parse them and index them so that we can properly retrieve them. We'll save them in a Vector Search endpoint and leverage it to guide the operators with the maintenance step and recommendations.

We'll use a Managed embedding index to make it simple. In this section we will:

1. Parse and save our PDF text in a Delta Table using Databricks AI Query `ai_parse_document`
2. Create a `Vector Search endpoint` (required to host your vector search index)
3. Create a `Vector Search Direct Index`  (the actual index)
4. Create a `Tool (UC function)` using our vector search 


### 2.1. Parse and save our PDF text
Let's start by parsing the maintenance guide documents, saved as pdf in our volume:

In [0]:
%sql
DROP TABLE IF EXISTS turbine_maintenance_guide;

CREATE TABLE turbine_maintenance_guide (
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  EAN STRING,
  weight STRING,
  component_type STRING,
  component_name STRING,
  full_guide STRING)
  TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
%sql
INSERT INTO turbine_maintenance_guide (EAN, weight, component_type, component_name, full_guide)

SELECT ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the EAN from the following text. Return only the EAN. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS EAN  -- Placeholder for the output column
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component weight from the following text. Return only the component weight. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS weight  -- Placeholder for the output column
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component type from the following text. Return only the component type. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS component_type  -- Placeholder for the output column 
    ,ai_query(
    'databricks-gemma-3-12b',
    CONCAT("Extract the component name from the following text. Return only the component name. \n\nText:", full_guide)    -- Placeholder for the prompt and input
    ) AS component_name  -- Placeholder for the output column
    ,full_guide 
FROM (
    -- Combine the content of all pages into a single string separated by new lines
    SELECT array_join(

            -- Transform each page struct in the array to just its 'content' field (extract text from each page)
            transform(
              parsed_document:document.pages::ARRAY<STRUCT<content:STRING>>, -- Array of page structs from the parsed document
              x -> x.content -- For each struct (page), extract the 'content' string
             ), 
             
             '\n' -- Join all extracted page contents with a newline character as the separator
             ) 
             
             AS full_guide
    FROM (
      -- Parse the document content
      SELECT ai_parse_document(content) AS parsed_document
      FROM READ_FILES("/Volumes/main/dbdemos_iot_turbine/turbine_raw_landing/maintenance_guide", format => 'binaryFile')
    )
);

SELECT * FROM turbine_maintenance_guide

id EAN weight component_type component_name full_guide 1 96356234 4014g Anemometer AeroSense VTX-220 # AeroSense VTX-220 Precision Anemometer

Professional Maintenance Manual - Wind Turbine Component

Part Type: Anemometer
EAN: 96356234
Compatible Turbine: Skylance XR550 Coastal Turbine Platform
Sensors Used: sensor_D, sensor_C
Dimensions: 364mm - 286mm
Weight: 4014g
Stock Zone: America/New_York

## Component Overview

The AeroSense VTX-220 is a precision optical-cup anemometer designed to operate in harsh marine and coastal environments.

It delivers high-frequency wind speed measurements to the turbine's main control unit via sensor_D and communicates backup diagnostics via sensor_C.

Equipped with a UV-resistant ABS housing and dual-bearing stainless steel shaft, it maintains sub-0.1s response latency in gusts up to 180 km/h.

## Recognizing Issues

Operators may notice discrepancies between actual weather data and turbine logs, delayed SCADA wind alerts, or sudden drops in power generation.

Typical warning signs include inconsistent wind-speed readings or visible physical obstruction of the rotor cups.

## Error Codes & Troubleshooting

ANM-100
Description: Low RPM detected despite wind presence. Likely mechanical blockage or bearing failure.

Resolution: Inspect the cup rotor for salt deposits or bird interference. Apply approved cleaning fluid and rotate manually to check bearing friction.

ANM-201

Description: Intermittent signal loss on sensor_D channel. Data irregularities exceed 15% over 10 minutes.

Resolution: Replace connector or test with alternate input pin. Ensure waterproof sealing and corrosion-free contact surface.

ANM-310

Description: Excessive vibration detected beyond 2.0g. Shaft imbalance or mounting flange fault suspected.

Resolution: Use vibration sensor diagnostics to assess severity. Tighten mounting base, and recalibrate pitch compensation in firmware settings.

## Recommended Maintenance Schedule

Inspect every 2,500 operational hours. Replace every 12,000 hours or if more than two error events occur in a 90-day period.

Preventative cleaning every 6 months in salt-air environments is strongly advised.

## Certified Maintenance Procedure

1. Engage turbine maintenance mode from SCADA dashboard. Confirm rotor lock and system isolation.

2. Access the rooftop nacelle via secured ladder or lift. Wear high-visibility PPE and fall arrest gear rated for 100kg minimum.

3. Locate the anemometer mast on the rear left corner of the nacelle shell. Document physical condition with photos before removal.
4. Disconnect the twin sensor cable junction box under the mast. Use an IP67-rated cap to cover open leads during maintenance.

5. Using a torque wrench, loosen the 3 mounting bolts (13mm) securing the anemometer base. Hold the unit from above to prevent fall.

6. Gently lift the AeroSense VTX-220 unit upward and inspect the shaft coupling area for corrosion or fatigue.

7. Clean the mounting flange area with alcohol-based surface cleaner and dry thoroughly using lint-free cloth.

8. Prepare the replacement unit by verifying serial number, firmware revision, and alignment pin compatibility.

9. Insert the VTX-220 anemometer onto the flange, align the guiding notch, and secure all bolts to 28 Nm torque.

10. Reconnect sensor_D and sensor_C to the color-coded terminal blocks in the junction box. Use dielectric grease on terminals.

11. Run a full sensor check using the Skylance Diagnostic Utility. Validate RPM, signal variance, and latency under simulated gust input.

12. Recalibrate the wind-speed baseline against local weather station data. Accept results only if variance < 1.2%.

13. Document replacement with timestamp, technician ID, and attach photos to maintenance log in the turbine management system. 2 50311982 9200g Spring-applied hydraulic brake caliper SBC-1300 # BrakeTech SBC-1300 Spring-Applied Brake Caliper

Professional Maintenance Manual - Rotor Braking System

Component Type: Spring-applied 

### 2.2. Creating the Vector Search endpoint

Let's create a new Vector search endpoint. You can also use the [UI under Compute](#/setting/clusters/vector-search) to directly create your endpoint.

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient(disable_notice=True)

if not endpoint_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME):
    vsc.create_endpoint(name=VECTOR_SEARCH_ENDPOINT_NAME, 
                        endpoint_type="STANDARD")

wait_for_vs_endpoint_to_be_ready(vsc, 
                                 VECTOR_SEARCH_ENDPOINT_NAME)


print(f"Endpoint named {VECTOR_SEARCH_ENDPOINT_NAME} is ready.")

Waiting for endpoint to be ready, this can take a few min... {'name': 'dbdemos_vs_endpoint', 'creator': 'kemalcan@berkeley.edu', 'creation_timestamp': 1754756481685, 'last_updated_timestamp': 1754756481685, 'endpoint_type': 'STANDARD', 'last_updated_user': 'kemalcan@berkeley.edu', 'id': '64e505c6-1f73-4537-88e1-0b16b4524388', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'dbdemos_vs_endpoint', 'creator': 'kemalcan@berkeley.edu', 'creation_timestamp': 1754756481685, 'last_updated_timestamp': 1754756481685, 'endpoint_type': 'STANDARD', 'last_updated_user': 'kemalcan@berkeley.edu', 'id': '64e505c6-1f73-4537-88e1-0b16b4524388', 'endpoint_status': {'state': 'PROVISIONING'}, 'num_indexes': 0}
Waiting for endpoint to be ready, this can take a few min... {'name': 'dbdemos_vs_endpoint', 'creator': 'kemalcan@berkeley.edu', 'creation_timestamp': 1754756481685, 'last_updated_timestamp': 1754756481685, 'endpoint


### 2.3 Creating the Vector Search Index

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/index_creation.gif?raw=true" width="600px" style="float: right; margin-left: 10px">

You can view your endpoint on the [Vector Search Endpoints UI](#/setting/clusters/vector-search). Click on the endpoint name to see all indexes that are served by the endpoint.

All we now have to do is to as Databricks to create the index on top of our table. The Delta Table will automatically be synched with the index.


Again, you can do that using your Unity Catalog UI, and selecting the turbine_maintenance_guide table in your Unity Catalog, and click on add a vector search. 

In [0]:
import databricks.sdk.service.catalog as c

# Where we want to store our index
vs_index_fullname = f"{catalog}.{db}.turbine_maintenance_guide_vs_index"

if not index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname):
  print(f"Creating index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  
  index = vsc.create_delta_sync_index(
    endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
    source_table_name=f"{catalog}.{db}.turbine_maintenance_guide",
    index_name=vs_index_fullname,
    pipeline_type="TRIGGERED",
    primary_key='id',
    embedding_source_column="full_guide",
    embedding_model_endpoint_name="databricks-gte-large-en"
  )
else:
  print(f"Grabbing existing index {vs_index_fullname} on endpoint {VECTOR_SEARCH_ENDPOINT_NAME}...")
  index = vsc.get_index(VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)

Creating index main.dbdemos_iot_turbine.turbine_maintenance_guide_vs_index on endpoint dbdemos_vs_endpoint...


In [0]:
exists = index_exists(vsc, VECTOR_SEARCH_ENDPOINT_NAME, vs_index_fullname)
display(spark.createDataFrame([(vs_index_fullname, exists)], ["index_name", "exists"]))

index_name,exists
main.dbdemos_iot_turbine.turbine_maintenance_guide_vs_index,true


### 2.4 Create our tool
Below, we utilize the _VECTOR\_SEARCH_ SQL function from Databricks to easily set up our maintenance reports retriever function. Our agent will utilize this function in the subsequent steps!

In [0]:
spark.sql("DROP FUNCTION IF EXISTS turbine_maintenance_guide_retriever")
spark.sql(f"""
CREATE OR REPLACE FUNCTION 
turbine_maintenance_guide_retriever(question STRING COMMENT 'Question to be answered from the turbine maintenance guides.')
RETURNS ARRAY<STRING>
LANGUAGE SQL
COMMENT 'This tool searches / retrieves the wind turbine maintenance guide for a given question'
RETURN (
  SELECT collect_list(full_guide) 
  FROM VECTOR_SEARCH(index => '{catalog}.{schema}.turbine_maintenance_guide_vs_index', query => question, num_results => 1) ) """)

DataFrame[]

In [0]:
%sql 
-- Let's test the tool we created
SELECT turbine_maintenance_guide_retriever('The VibeGuard TVS-950 is giving me an error code TVS-001.') AS reports

reports
"List(# VibeGuard TVS-950 Tower Vibration Sensor Professional Maintenance Manual - Tower Structural Monitoring ## Component Type: Tri-axis vibration sensor EAN: 77630941 Compatible Turbine Model: ZephyrTech ZT1000 Coastal-Class Turbine Dimensions: 75mm × 75mm Weight: 620g Sensor Interfaces: sensor_Z, sensor_X, sensor_Y Stock Location: Portugal/Porto ## Component Overview The VibeGuard TVS-950 is a tri-axis MEMS-based vibration sensor installed mid-tower to monitor lateral, vertical, and torsional vibrations of the wind turbine structure. It captures tower resonance profiles and harmonic peaks to detect foundation shifts, loose bolts, and structural fatigue. This sensor outputs real-time telemetry to SCADA and anomaly detection systems, helping prevent long-term mechanical degradation. It interfaces with sensors Z (vertical), X (lateral), and Y (torsional) using shielded cabling and provides both raw vibration signature and FFT summaries. ## Vibration Anomalies and Failure Indicators - Abnormal SCADA vibration levels in idle or operating mode - FFT frequency anomalies not matching tower resonance - Signal loss or thermal fault warnings from sensor interface - Structural inspection triggers based on peak force readings # Error Codes and Troubleshooting Guidelines TVS-001 Description: Vertical vibration (Z) exceeds 0.6g RMS for >10s. Resolution: Inspect tower base and anchor bolts. Check for nearby seismic or storm activity. Tighten structural joints. TVS-016 Description: Sensor_X signal flatlined - no motion detected. Resolution: Verify sensor mounting bracket is tight. Check cable connection. Replace sensor if MEMS axis is non-responsive. TVS-029 Description: Harmonic peak detected at 12 Hz - outside known resonance profile. Resolution: Inspect nacelle yaw system and tower top for oscillation source. Check blade balance and gearbox mounts. TVS-044 Description: Sensor_Y FFT checksum mismatch. Resolution: Restart SCADA FFT module. Replace sensor if checksum mismatch persists after diagnostics reboot. TVS-057 Description: Temperature on sensor PCB >80°C sustained. Resolution: Inspect cooling airflow or proximity to electrical heater source. Relocate sensor if ambient exceeds spec. TVS-072 Description: Cable impedance out of range - potential shielding failure. Resolution: Replace signal cable from sensor to controller. Check for physical damage or improper grounding. TVS-093 Description: Sensor Z reports peak >2.5g - suspected structural event. Resolution: Trigger full structural inspection. Check SCADA for simultaneous brake/yaw events. Log impact and sensor history. ## Recommended Inspection Interval Check sensor condition and FFT profile quarterly, especially after storms. Replace after 15,000 hours or if g-force anomalies are recorded. ## Professional Replacement Procedure 1. Disable sensor power supply from SCADA diagnostics panel. 2. Access mid-tower service platform. Confirm lockout tagout safety procedure is active. 3. Locate vibration sensor housing. Use PPE due to possible oil residue or confined space. 4. Disconnect signal cabling for axes Z, X, and Y. Label connectors for reassembly. 5. Remove mounting screws and detach sensor from baseplate. Inspect for corrosion or crack near mounts. 6. Visually inspect sensor body for deformation. Gently clean with antistatic cloth. 7. Install new VibeGuard TVS-950. Use Loctite threadlocker and torque to 1.5 Nm. 8. Reconnect signal cables. Use EMI shielding sleeves if available. Confirm snug fit. 9. Re-enable power and verify initialization LEDs blink correctly. 10. Run vibration baseline scan from SCADA and verify RMS values in idle mode. 11. Compare SCADA FFT data with historical signature for sensor X, Y, and Z axes. 12. Document sensor serial, mount location, and install time. Upload to digital maintenance log. 13. Check torque on all tower bolts in vicinity. Monitor vibration for 2-hour post-swap period. 14. Log replacement in SCADA ticket and archive vibration plot ima

## Exploring Mosaic AI Tools in Unity Catalog

Our tools are ready! 

You can now view the UC function tools in Catalog Explorer. Click **Catalog** in the sidebar. In the Catalog Explorer, navigate to your catalog and schema. 

The UC function tools appears under **Functions**. 

<img src="https://github.com/Datastohne/demo/blob/main/Screenshot%202024-09-18%20at%2016.24.24.png?raw=true"/>

## What’s next: test your Agents with Databricks Playground

Now that we have our AI Tools ready and registered in Unity Catalog, we can compose them into an agent system that generates maintenance work orders using the Mosaic AI agent framework.

Open the [05.2-agent-creation-guide]($./05.2-agent-creation-guide) notebook to create and deploy the system.